In [1]:
import time

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [2]:
url = 'https://www.saramin.co.kr/zf_user/'
browser = webdriver.Chrome()
browser.get(url)

In [3]:
browser.find_element(By.CLASS_NAME, 'recruit').click()
seoul_all = browser.find_element(By.XPATH, '//*[@id="loc_mcd_101000"]') ## 서울 전체
browser.execute_script("arguments[0].click();", seoul_all)

gyeonggi_all = browser.find_element(By.XPATH, '//*[@id="depth1_btn_102000"]/button') ## 경기
browser.execute_script("arguments[0].click();", gyeonggi_all)

browser.find_element(By.CLASS_NAME, 'job_category_section').click()
browser.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/div/div[2]/div[2]/div[1]/button[6]').click()
browser.find_element(By.XPATH, '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[2]/dd/button[6]').click()
browser.find_element(By.XPATH, '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[2]/dd/button[7]').click()
browser.find_element(By.XPATH, '//*[@id="search_btn"]').click()

In [5]:
container = browser.find_element(By.CLASS_NAME, 'list_body')
blocks = container.find_elements(By.CLASS_NAME, 'list_item')
blocks.pop(21)
print(len(blocks))

50


In [19]:
current_page_data = []
for block in blocks:
    try:
        have_id = block.get_attribute('id')
        company_name_box = block.find_element(By.CLASS_NAME, 'company_nm')
        company_name = company_name_box.find_element(By.CLASS_NAME, 'str_tit').text

        recurit_box = block.find_element(By.CLASS_NAME, 'notification_info')
        title_box = recurit_box.find_element(By.CLASS_NAME, 'str_tit')
        title = title_box.text
        link = title_box.get_attribute('href')

        detail_fields_box = block.find_element(By.CLASS_NAME, 'job_meta').find_element(By.CLASS_NAME, 'job_sector')
        detail_fields = detail_fields_box.find_elements(By.TAG_NAME, 'span')
        detail_fields = ','.join([x.text for x in detail_fields])

        option_box = block.find_element(By.CLASS_NAME, 'recruit_info')
        options = option_box.find_elements(By.TAG_NAME, 'p')
        options = ','.join([option.text.replace(' · ', '') for option in options])

        data = {'회사명' : company_name, '채용공고 제목' : title, '채용공고 세부 사항' : options, '기술 세부 사항' : detail_fields, '링크' : link}
        current_page_data.append(data)
        
    except:
        continue

    
    print(options)

서울전체 외,신입경력정규직,대학교(4년)↑
경기 남양주시,경력 3년↑정규직,대학교(4년)↑
서울전체 외,신입경력정규직,대학(2,3년)↑
서울전체 외,신입경력정규직,대학교(4년)↑
서울전체 외,경력 1년↑정규직,학력무관
서울전체 외,신입경력정규직 외,대학(2,3년)↑
경기 안양시 외,경력정규직,대학교(4년)↑
서울전체 외,경력무관교육생,학력무관
서울 금천구,신입경력정규직 외,대학교(4년)↑
서울 서초구,신입경력정규직 외,석사↑
서울 강남구,경력 2년↑정규직,학력무관
경기 부천시 외,경력정규직,학력무관
서울 강남구,2 ~ 10년정규직,대학(2,3년)↑
서울 종로구,경력무관정규직 외,대학교(4년)↑
서울 송파구,10 ~ 15년정규직,학력무관
서울 노원구,경력무관정규직,대학교(4년)↑
경기전체 외,3 ~ 13년정규직,대학교(4년)↑
서울 금천구,신입경력정규직,대학교(4년)↑
서울 양천구,경력 3년↑정규직,대학(2,3년)↑
서울전체 외,5 ~ 10년정규직,학력무관
서울전체 외,경력 5년↑프리랜서,학력무관
경기 성남시 분당구 외,신입경력정규직 외,고졸↑
경기 수원시 영통구,신입경력정규직,대학교(4년)↑
경기 안양시 동안구,경력 3년↑정규직,학력무관
경기 성남시 분당구,신입경력정규직,대학교(4년)↑
경기 성남시 분당구 외,경력무관정규직,대학(2,3년)↑
서울전체 외,2 ~ 5년정규직,고졸↑
서울 광진구 외,5 ~ 10년정규직,대학교(4년)↑
경기 화성시,경력무관정규직,대학교(4년)↑
서울 구로구,경력 5년↑정규직,석사↑
대전 유성구 외,경력무관정규직,학력무관
경기 의왕시,3 ~ 5년정규직,학력무관
서울 강남구,신입경력정규직,대학교(4년)↑
경기 성남시 분당구,경력 7년↑정규직,석사↑
경기 고양시 일산서구,신입경력정규직,대학교(4년)↑
서울 강남구 외,신입정규직,대학교(4년)↑
서울 강남구,신입경력정규직,석사↑
서울 관악구,경력 2년↑정규직,학력무관
서울 송파구,2 ~ 10년정규직,대학교(4년)↑
서울전체 외,1 ~ 5년정규직,학력무관
경기전체 외,신입경력정규직,대학교(4년

In [ ]:
## page1 url :https://www.saramin.co.kr/zf_user/jobs/list/domestic?loc_mcd=101000%2C102000&cat_kewd=108%2C109&panel_type=&search_optional_item=n&search_done=y&panel_count=y&preview=y
## page2 url : https://www.saramin.co.kr/zf_user/jobs/list/domestic?page=2&loc_mcd=101000%2C102000&cat_kewd=108%2C109&search_optional_item=n&search_done=y&panel_count=y&preview=y&isAjaxRequest=0&page_count=50&sort=RL&type=domestic&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=2#searchTitle